# Qcodes example with InstrumentGroup driver

This notebooks explains how to use the `InstrumentGroup` driver.

## About
The goal of the `InstrumentGroup` driver is to combine several instruments as submodules into one instrument. Typically, this is meant to be used with the `DelegateInstrument` driver. An example usage of this is to create an abstraction for devices on a chip.

## Usage
The way it's used is mainly by specifying an entry in the station YAML. For instance, to create a Chip that has one or more Devices on it that point to different source parameters.

In [1]:
%%writefile example.yaml

instruments:
  mdac:
    type: qcodes.tests.instrument_mocks.MockDAC
    init:
      num_channels: 2

  lockin1:
    type: qcodes.tests.instrument_mocks.MockLockin

  lockin2:
    type: qcodes.tests.instrument_mocks.MockLockin

  MockChip_123:
    type: qcodes.instrument.delegate.InstrumentGroup
    init:
      submodules:
        device1:
            type: qcodes.instrument.delegate.DelegateInstrument
            gate:
              - mdac.ch01.voltage
            source:
              - lockin1.frequency
              - lockin1.amplitude
              - lockin1.phase
              - lockin1.time_constant
            drain:
              - lockin1.X
              - lockin1.Y
        device2:
            type: qcodes.instrument.delegate.DelegateInstrument
            gate:
              - mdac.ch02.voltage
            source:
              - lockin2.frequency
              - lockin2.amplitude
              - lockin2.phase
              - lockin2.time_constant
            drain:
              - lockin2.X
              - lockin2.Y

      set_initial_values_on_load: true
      initial_values:
        device1:
          gate.step: 5e-4
          gate.inter_delay: 12.5e-4

Writing example.yaml


In [1]:
import qcodes as qc

In [2]:
station = qc.Station(config_file="example.yaml")
lockin1 = station.load_lockin1()
lockin2 = station.load_lockin2()
mdac = station.load_mdac()
chip = station.load_MockChip_123(station=station)

device1_gate step 0.0005
device1_gate inter_delay 0.00125


In [3]:
chip.device1.gate()

0.0

In [4]:
mdac.ch01.voltage()

0.0

In [5]:
chip.device1.gate(1.0)
chip.device1.gate()

1.0

In [6]:
mdac.ch01.voltage()

1.0

In [7]:
chip.device1.source()

source(frequency=125.0, amplitude=0.0, phase=0.0, time_constant=0.001)

In [8]:
chip.device1.drain()

drain(X=0.001, Y=1e-05)